In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.config.list_physical_devices('GPU')

tf.test.is_built_with_cuda()

In [ ]:
import cv2

DataDir = "C:/Users/andyl/Downloads/ngdgg79rzb-1/plantvillage/Preprocessed data/Preprocessed data"
Categories = ["Bacterial_spot227", "Early_blight227", "healthy227", "Late_blight227", "Leaf_Mold227", 
              "Septoria_leaf_spot227", "Target_Spot227", "Tomato_mosaic_virus227", 
              "Tomato_Yellow_Leaf_Curl_Virus227", "Two-spotted_spider_mite227"]

for category in Categories:
    path = os.path.join(DataDir, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
        plt.imshow(img_array, cmap="gray")
        plt.show()
        break
    break

In [ ]:
training_data = []

def create_training_data():
    for category in Categories:
        path = os.path.join(DataDir, category)
        class_num = Categories.index(category) 
        for img in os.listdir(path):
            try: 
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR) 
                training_data.append([img_array, class_num])
            except Exception as e: 
                pass

create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
import random

random.shuffle(training_data)

In [ ]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

Img_Size = 227

X = np.array(X).reshape(-1, Img_Size, Img_Size,1)

In [ ]:
import time
Name = "TomatoDisease-cnn-64x2-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir='logs/{}'.format(Name))

X = np.array(X/255.0)
y = np.array(y)

model = Sequential()
model.add(Conv2D(16, (3,3), input_shape = X.shape[1:]))
# rectified linear activation
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3)))
# rectified linear activation
model.add(Activation("relu"))
model.add(MaxPooling2D())


model.add(Flatten()) # converts 3D features into 1D feature
model.add(Dense(32))
model.add(Activation("relu"))

# output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2, callbacks=[tensorboard])

# save the trained model
model.save('TomatoDisease-CNN.model')